In [45]:
from pynq import Overlay
from pynq import Clocks
from time import sleep
from IPython.display import clear_output

# indicando o bitstream
ol = Overlay('/home/xilinx/pynq/overlays/motion_detector/design_1.bit')
ol.ip_dict

{'motion_detector': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xb30d5470>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'motion_detector',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S00_AXI',
  'parameters': {'C_S00_AXI_ADDR_WIDTH': '6',
   'C_S00_AXI_BASEADDR': '0x43C00000',
   'C_S00_AXI_DATA_WIDTH': '32',
   'C_S00_AXI_HIGHADDR': '0x43C0FFFF',
   'Component_Name': 'design_1_motion_detector_0',
   'EDK_IPTYPE': 'PERIPHERAL'},
  'phys_addr': 1136656384,
  'registers': {},
  'state': None,
  'type': 'xilinx.com:user:motion_detector_and_AXI_Lite_1_3:1.3'},
 'processing_system7_0': {'device': <pynq.pl_server.device.XlnkDevice at 0xb30d5470>,
  'driver': pynq.overlay.DefaultIP,
  'parameters': {'C_DM_WIDTH': '4',
   'C_DQS_WIDTH': '4',
   'C_DQ_WIDTH': '32',
   'C_EMIO_GPIO_WIDTH': '64',
   'C_EN_EMIO_ENET0': '0',
   'C_EN_EMIO_ENET1': '0',
   'C_EN_EMIO_PJTAG': '0',
   'C_EN_EMIO_TRACE': '0',
   'C_FCLK_CLK0_BUF': 'TRUE',
   'C_FCLK_CLK1_BUF': 'FALSE

In [46]:
# Veriricando os clks do PS7
print(f'CPU:   {Clocks.cpu_mhz:.6f}MHz')
print(f'FCLK0: {Clocks.fclk0_mhz:.6f}MHz')
print(f'FCLK1: {Clocks.fclk1_mhz:.6f}MHz')
print(f'FCLK2: {Clocks.fclk2_mhz:.6f}MHz')
print(f'FCLK3: {Clocks.fclk3_mhz:.6f}MHz')

CPU:   650.000000MHz
FCLK0: 100.000000MHz
FCLK1: 100.000000MHz
FCLK2: 100.000000MHz
FCLK3: 100.000000MHz


In [47]:
# definindo o endereço de cada registrador que vem do IP
# slv_reg0 (0x0)  -- Destinado para o Start (INPUT)
start_addr = 0x0 
# slv_reg1 (0x4)  -- Destinado para o pixelAvailable (INPUT)
pixelAvailable_addr = 0x4
# slv_reg2 (0x8)  -- Destinado para o pixelValue (INPUT)
pixelValue_addr = 0x8
# slv_reg3 (0xc) -- Destinado para o nextPixel (OUTPUT)
nextPixel_addr = 0xc
# slv_reg4 (0x10) -- Destinado para o pixelPosition (OUTPUT)
pixelPosition_addr = 0x10
# slv_reg5 (0x14) -- Destinado para o pixelAccumulator (OUTPUT)
pixelAccumulator_addr = 0x14
# slv_reg6 (0x18) -- Destinado para o motionAccumulator (OUTPUT)
am_addr = 0x18 
# slv_reg7 (0x1c) -- Destinado para o ready (OUTPUT)
ready_addr = 0x1c 
# slv_reg8 (0x20) -- Destinado para o result (OUTPUT)
result_addr = 0x20 

In [49]:
# Fazendo Download do IP
ol.download()
print(f'Overlay is loaded: {ol.is_loaded()} ({ol.timestamp})')

# define um objeto python para lidar com o IP motion_detector
md = ol.motion_detector
#md?
#md.register_map

# Inicializa o contador de pixel para controle das operações com o IP
pixel_counter = 0
pixelPosition_PS = 0

# Abre o arquivo de pixels
fd = open('Movendo.txt')

# Dá o start no IP de detecção de movimento
md.write(start_addr, 1)
#print(f'start = {md.read(start_addr)}')
sleep(1)


inLoop = True
while inLoop:
    
    # Verifica os registradores de saída do Motion Detector
    pixelPosition_PL = md.read(pixelPosition_addr)
    pixelAvailable = md.read(pixelAvailable_addr)
    pixelValue = md.read(pixelValue_addr)
    pixelAccumulator = md.read(pixelAccumulator_addr)
    nextPixel = md.read(nextPixel_addr)
    am = md.read(am_addr)
    ready = md.read(ready_addr)
    result = md.read(result_addr)
    
    '''
    clear_output(wait=True)
    print(f'pixel_counter = {pixel_counter}\t   PL/PS \n \
            pixelPosition: {pixelPosition_PL}/{pixelPosition_PS} \n \
            nextPixel: {nextPixel} \n \
            pixelValue = {pixelValue}, reg[0] = {pixelAccumulator}, am = {am} \n')
    '''        
    
    if nextPixel == 1 and pixelPosition_PL == pixelPosition_PS and ready != 1:
        
        clear_output(wait=True)
        print(f'pixel_counter = {pixel_counter}\t   PL/PS \n \
                pixelPosition: {pixelPosition_PL}/{pixelPosition_PS} \n \
                pixelValue = {pixelValue}, reg[0] = {pixelAccumulator}, am = {am} \n')
        
        # Lê uma linha do arquivo de pixel 
        line = fd.readline()
        if not line:
            inLoop = False
            print('End of file')
            break
        #print(line.rstrip())

        # Transforma o valor lido de binário para inteiro    
        pixel = int(line, base = 2)
        #print(pixel)

        # Escreve o valor do pixel no barramento do IP
        md.write(pixelValue_addr, pixel)
    
        # Informa que o novo pixel está disponível
        md.write(pixelAvailable_addr, 1)
    
        # Itera o contador de pixel
        pixel_counter += 1
        pixelPosition_PS += 1
        if pixelPosition_PS == 16: 
            pixelPosition_PS = 0 
    
    elif pixelPosition_PL != pixelPosition_PS:
        # Informa que o novo pixel está disponível
        md.write(pixelAvailable_addr, 1)
        
    elif ready == 1:
        print(f'pixel_counter = {pixel_counter}\t   PL/PS \n \
                pixelPosition: {pixelPosition_PL}/{pixelPosition_PS} \n \
                pixelValue = {pixelValue}, reg[0] = {pixelAccumulator}, am = {am} \n \
                ready = {ready}, result = {result} \n')

        
    #sleep(1)

pixel_counter = 170	   PL/PS 
                 pixelPosition: 10/10 
                 pixelValue = 209, reg[0] = 3592, am = 0 



KeyboardInterrupt: 